In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import layers
from tensorflow.keras import Model

In [57]:
train_dir = 'C:/Users/Administrator/Desktop/Final Project/data_croped_raw/train'
validation_dir = 'C:/Users/Administrator/Desktop/Final Project/data_croped_raw/val/manipulated_sequences/NeuralTextures'

In [3]:
test_dir = 'C:/Users/Administrator/Desktop/Final Project/data_croped_raw/test'

In [4]:
total = 0
for root, dirs, files in os.walk('C:/Users/Administrator/Desktop/Final Project/data_croped_raw/test/original_sequences/actors'):
   total += len(files)

NameError: name 'os' is not defined

In [ ]:
total

In [48]:
train_datagen = ImageDataGenerator( rescale = 1.0/255.,
                                  
                                  )
test_datagen  = ImageDataGenerator( rescale = 1.0/255,
                                  
                                  )

In [6]:
testing_datagen = ImageDataGenerator( rescale=1.0/255)

In [35]:
train_generator = train_datagen.flow_from_directory(train_dir,
                                                    batch_size=128,
                                                    class_mode='binary',
                                                    target_size = (224, 224))

Found 5012 images belonging to 72 classes.


In [58]:
validation_generator =  test_datagen.flow_from_directory(validation_dir,
                                                         batch_size=128,
                                                         class_mode='binary',              
                                                         target_size = (224, 224))

Found 812 images belonging to 14 classes.


In [9]:
test_generator =  testing_datagen.flow_from_directory(test_dir,
                                                         batch_size=128,
                                                         class_mode  = 'binary',
                                                         shuffle=False,
                                                         target_size = (224, 224))

Found 7405 images belonging to 2 classes.


In [53]:
from tensorflow.keras.applications import InceptionV3
pre_trained_model = InceptionV3(input_shape = (224, 224, 3), 
                                include_top = False, 
                                weights = 'imagenet')

#for layer in pre_trained_model.layers:
#    layer.trainable = False
pre_trained_model.trainable=True
last_layer = pre_trained_model.get_layer('mixed3')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 12, 12, 768)


In [54]:
len(pre_trained_model.layers)

311

In [55]:
pre_trained_model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_106 (Conv2D)             (None, 111, 111, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_106 (BatchN (None, 111, 111, 32) 96          conv2d_106[0][0]                 
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 111, 111, 32) 0           batch_normalization_106[0][0]    
_______________________________________________________________________________________

In [56]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import layers,models
from tensorflow.keras import Model
def create_model():
    
 
  # Flatten the output layer to 1 dimension
  x = layers.Flatten()(last_output)

  x = layers.Dropout(0.4)(x) 

  # Add a fully connected layer with 1,024 hidden units and ReLU activation
  x = layers.Dense(128, activation='relu')(x)
  # Add a dropout rate of 0.2
  x = layers.Dropout(0.4)(x) 
  # Add a fully connected layer with 1,024 hidden units and ReLU activation
  x = layers.Dense(64, activation='relu')(x)
  # Add a dropout rate of 0.2
  x = layers.Dropout(0.2)(x) 

  # Add a final sigmoid layer for classification
  x = layers.Dense  (1, activation='sigmoid')(x) 
  model = Model( pre_trained_model.input, x) 

  model.compile(optimizer = Adam(), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

  return model


In [57]:
def create_model2():
  # Flatten the output layer to 1 dimension

  flatten_layer = layers.Flatten()
  drop_layer_1 = layers.Dropout(0.4)

  # Add a fully connected layer with 1,024 hidden units and ReLU activation
  dense_layer_1 = layers.Dense(1024, activation='relu')
  # Add a dropout rate of 0.2
  drop_layer_2 = layers.Dropout(0.2)
  dense_layer_2 = layers.Dense(512, activation='relu')
    
  drop_layer_3= layers.Dropout(0.2)

  dense_layer_3 = layers.Dense(128, activation='relu')
    
  drop_layer_4= layers.Dropout(0.2)
  
  # Add a final sigmoid layer for classification
  prediction_layer = layers.Dense  (1, activation='sigmoid')

  model = models.Sequential([
      pre_trained_model,
      flatten_layer,
#      drop_layer_1,
#      dense_layer_1,
#      drop_layer_2,
#      dense_layer_2,
#      drop_layer_3,
#      dense_layer_3,
#      drop_layer_4,
      prediction_layer
  ])

  model.compile(optimizer = Adam(), 
              loss = 'binary_crossentropy', 
              metrics = ['accuracy'])

  return model

In [58]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))


Found GPU at: /device:GPU:0


In [59]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  1


In [60]:
model=create_model()
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_106 (Conv2D)             (None, 111, 111, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_106 (BatchN (None, 111, 111, 32) 96          conv2d_106[0][0]                 
__________________________________________________________________________________________________
activation_94 (Activation)      (None, 111, 111, 32) 0           batch_normalization_106[0][0]    
____________________________________________________________________________________________

In [61]:
import os
checkpoint_path = 'C:/Users/Administrator/Desktop/Final Project/mostafa/training_1'
checkpoint_dir = os.path.dirname(checkpoint_path)

# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_best_only=True,
                                                 monitor='val_accuracy',
                                                 mode='max',
                                                 verbose=1)

In [62]:
callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy',restore_best_weights = True, patience=3)

In [63]:
#tf.debugging.set_log_device_placement(True)
with tf.device('GPU:0'):
  history = model.fit(
              train_generator,
              validation_data = validation_generator,
              #edit 
              steps_per_epoch = 306,
              epochs = 10,
              #edit
              validation_steps = 59,
              callbacks=[cp_callback,callback],
              verbose = 1)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 306 steps, validate for 59 steps
Epoch 1/10
305/306 [============================>.] - ETA: 0s - loss: 0.4256 - accuracy: 0.8900
Epoch 00001: val_accuracy improved from -inf to 0.87924, saving model to C:/Users/Administrator/Desktop/Final Project/mostafa/training_1
INFO:tensorflow:Assets written to: C:/Users/Administrator/Desktop/Final Project/mostafa/training_1\assets
306/306 [==============================] - 351s 1s/step - loss: 0.4243 - accuracy: 0.8903 - val_loss: 0.6815 - val_accuracy: 0.8792
Epoch 2/10
305/306 [============================>.] - ETA: 0s - loss: 0.0559 - accuracy: 0.9829
Epoch 00002: val_accuracy improved from 0.87924 to 0.90651, saving model to C:/Users/Administrator/Desktop/Final Project/mostafa/training_1
INFO:tensorflow:Assets written to: C:/Users/Administrator/Desktop/Final Project/mostafa/training_1\assets
306/306 [==============================] - 347s 1s/step - loss: 0.0558 - accuracy: 0.9829 - va

In [10]:
from tensorflow.keras.models import load_model

In [11]:
model=load_model('best_model_raw.h5')

In [13]:
model.evaluate(train_generator)

  ...
    to  
  ['...']
48/48 [==============================] - 20s 415ms/step - loss: -904.1640 - accuracy: 0.0083


[-904.164000193278, 0.00832789]

In [15]:
import numpy as np

In [60]:
Y_pred = model.predict_generator(validation_generator)
y_pred = np.array([1 * (x[0]>=0.5) for x in Y_pred])
print(len(y_pred[y_pred==0])/len(y_pred)*100)

77.70935960591133


In [130]:
#Confution Matrix and Classification Report
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import numpy as np
Y_pred = model.predict_generator(test_generator)
y_pred = [1 * (x[0]>=0.5) for x in Y_pred]

cm = confusion_matrix(test_generator.classes, y_pred)
print('Confusion Matrix')
print(cm)
target_names=['fake','real']
print(classification_report(test_generator.classes, y_pred, target_names=target_names))


Confusion Matrix
[[4200  102]
 [  66 3037]]
              precision    recall  f1-score   support

        fake       0.98      0.98      0.98      4302
        real       0.97      0.98      0.97      3103

    accuracy                           0.98      7405
   macro avg       0.98      0.98      0.98      7405
weighted avg       0.98      0.98      0.98      7405



In [73]:
#model.save('best_model_raw.h5')

In [79]:
#model.save('model_raw_93.h5')

In [181]:
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']

val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

ImportError: cannot import name 'artist' from 'matplotlib' (C:\Users\Administrator\anaconda3\envs\tensor-gpu\lib\site-packages\matplotlib\__init__.py)

In [156]:
validation_generator.classes

array([0, 0, 0, ..., 1, 1, 1])

In [96]:
import matplotlib.pyplot as plt
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=validation_generator.classes)

disp.plot(cmap=plt.cm.Blues)
plt.show()

ImportError: cannot import name 'artist' from 'matplotlib' (C:\Users\Administrator\anaconda3\envs\tensor-gpu\lib\site-packages\matplotlib\__init__.py)

In [165]:
os.listdir(checkpoint_dir)


['.ipynb_checkpoints',
 'faceforensics.ipynb',
 'faceforensics2.ipynb',
 'Inception_youssef.ipynb',
 'model.ipynb',
 'model82.h5',
 'model82_m.h5',
 'model82_weights.h5',
 'myenv',
 'training_1',
 'venv']